In [1]:
import numpy as np
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Conv2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
import warnings
warnings.filterwarnings("ignore")

In [2]:
# re-size all the images to this
IMAGE_SIZE = [256, 256]

train_path = '/kaggle/input/final-year-2/Data/train_f_images'
valid_path = '/kaggle/input/final-year-2/Data/train_images'

In [3]:
from keras.applications.vgg16 import VGG16
SIZE = 224

VGG = VGG16(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))
VGG_fc = VGG16(weights='imagenet', include_top=True, input_shape=(SIZE, SIZE, 3))

VGG_model = Sequential()
VGG_model_fc = Sequential()

for layer in VGG.layers:
    VGG_model.add(layer) 
    
for layer in VGG_fc.layers[:-1]:
    VGG_model_fc.add(layer) 
    
# Freeze the layers    
for layer in VGG_model.layers:
    layer.trainable=False
    
for layer in VGG_model_fc.layers:
    layer.trainable=False

2022-12-03 10:33:56.620202: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-03 10:33:56.711258: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-03 10:33:56.712159: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-03 10:33:56.713943: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

553476096/553467096 [==============================] - 3s 0us/step


In [4]:
VGG_model_fc.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [5]:
import os
import cv2
import glob
#training and labes numpy array conversion
train_images=[]
train_labels=[]

for file in os.listdir('/kaggle/input/final-year-2/Data/train_f_images/diseased'):
    if file[0] == 'd':
        train_labels.append('diseased')
    else:
        train_labels.append('healthy')
    img=cv2.imread(os.path.join('/kaggle/input/final-year-2/Data/train_f_images/diseased',file))
    img=cv2.resize(img, (SIZE,SIZE))
    train_images.append(img)
    
for file in os.listdir('/kaggle/input/final-year-2/Data/train_f_images/healthy'):
    if file[0] == 'd':
        train_labels.append('diseased')
    else:
        train_labels.append('healthy')
    img=cv2.imread(os.path.join('/kaggle/input/final-year-2/Data/train_f_images/healthy',file))
    img=cv2.resize(img, (SIZE,SIZE))
    train_images.append(img)

    
train_images=np.array(train_images)
train_labels=np.array(train_labels)

In [6]:
#training and labes numpy array conversion
test_images=[]
test_labels=[]

for file in os.listdir('/kaggle/input/final-year-2/Data/test_images/diseased'):
    if file[0] == 'd':
        test_labels.append('diseased')
    else:
        test_labels.append('healthy')
    img=cv2.imread(os.path.join('/kaggle/input/final-year-2/Data/test_images/diseased',file))
    img=cv2.resize(img, (SIZE,SIZE))
    test_images.append(img)
    
for file in os.listdir('/kaggle/input/final-year-2/Data/test_images/healthy'):
    if file[0] == 'd':
        test_labels.append('diseased')
    else:
        test_labels.append('healthy')
    img=cv2.imread(os.path.join('/kaggle/input/final-year-2/Data/test_images/healthy',file))
    img=cv2.resize(img, (SIZE,SIZE))
    test_images.append(img)

    
test_images=np.array(test_images)
test_labels=np.array(test_labels)

In [7]:
frature_extractor=VGG_model_fc.predict(train_images)

2022-12-03 10:34:13.862382: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-12-03 10:34:14.933822: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


In [8]:
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report,plot_confusion_matrix
import keras.utils
from tensorflow.keras.utils import to_categorical
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(train_labels)
train_labels=label_encoder.transform(train_labels)
frature_extractor_test=VGG_model_fc.predict(test_images)
test_labels=label_encoder.transform(test_labels)

In [9]:
from sklearn.naive_bayes import GaussianNB
gb = GaussianNB()
gb.fit(frature_extractor,train_labels)
y_pred = gb.predict(frature_extractor)
print("train accuracy:{}".format(accuracy_score(train_labels,y_pred)))
y_pred = gb.predict(frature_extractor_test)
print("accuracy:{}".format(accuracy_score(test_labels,y_pred)))

train accuracy:0.8506912442396314
accuracy:0.7276190476190476


In [10]:
from sklearn.tree import DecisionTreeClassifier
dc = DecisionTreeClassifier()
dc.fit(frature_extractor,train_labels)
y_pred = dc.predict(frature_extractor)
print("train accuracy:{}".format(accuracy_score(train_labels,y_pred)))
y_pred = dc.predict(frature_extractor_test)
print("accuracy:{}".format(accuracy_score(test_labels,y_pred)))

train accuracy:1.0
accuracy:0.6780952380952381


In [11]:
from sklearn.ensemble import RandomForestClassifier
rc = RandomForestClassifier(n_estimators = 50)
rc.fit(frature_extractor,train_labels)
y_pred = rc.predict(frature_extractor)
print("train accuracy:{}".format(accuracy_score(train_labels,y_pred)))
y_pred = rc.predict(frature_extractor_test)
print("accuracy:{}".format(accuracy_score(test_labels,y_pred)))

train accuracy:1.0
accuracy:0.758095238095238


In [12]:
from sklearn.linear_model import LogisticRegression
lc = LogisticRegression()
lc.fit(frature_extractor,train_labels)
y_pred = lc.predict(frature_extractor)
print("train accuracy:{}".format(accuracy_score(train_labels,y_pred)))
y_pred = lc.predict(frature_extractor_test)
print("accuracy:{}".format(accuracy_score(test_labels,y_pred)))

train accuracy:1.0
accuracy:0.6704761904761904


In [13]:
import xgboost
xgb = xgboost.XGBRFClassifier()
xgb.fit(frature_extractor,train_labels)
y_pred = xgb.predict(frature_extractor)
print("train accuracy:{}".format(accuracy_score(train_labels,y_pred)))
y_pred = xgb.predict(frature_extractor_test)
print("test accuracy:{}".format(accuracy_score(test_labels,y_pred)))

train accuracy:0.9917050691244239
test accuracy:0.8342857142857143


In [14]:
from sklearn.model_selection import KFold,StratifiedKFold
kf = KFold(n_splits = 5, random_state = 1, shuffle = True)

In [15]:
preds = []
fold = StratifiedKFold(n_splits=5,shuffle=True,random_state=42)
i = 1
for train_id,test_id in fold.split(frature_extractor,train_labels):
        #training and testing data
        print("For Split {}".format(i))
        xtrain ,xtest = frature_extractor[train_id],frature_extractor[test_id]
        ytrain,ytest = train_labels[train_id],train_labels[test_id]
        rc = RandomForestClassifier()
        rc.fit(xtrain,ytrain)
        y_pred = rc.predict(xtrain)
        print("train accuracy:{}".format(accuracy_score(ytrain,y_pred)))
        y_pred = rc.predict(xtest)
        print("test accuracy:{}".format(accuracy_score(ytest,y_pred)))
        y_pred = rc.predict(frature_extractor_test)
        preds.append(y_pred)
        i+=1
preds = np.mean(np.column_stack(preds), axis=1)
new_preds = []
for x in preds:
    if x>=0.5:
        new_preds.append(1)
    else:
        new_preds.append(0)
print("Total Test Accuracy:{}".format(accuracy_score(new_preds,test_labels)))

For Split 1
train accuracy:1.0
test accuracy:0.8847926267281107
For Split 2
train accuracy:1.0
test accuracy:0.9170506912442397
For Split 3
train accuracy:1.0
test accuracy:0.8617511520737328
For Split 4
train accuracy:1.0
test accuracy:0.8847926267281107
For Split 5
train accuracy:1.0
test accuracy:0.880184331797235
Total Test Accuracy:0.7657142857142857


In [16]:
models = 1
import numpy as np
import pandas as pd 
import optuna 
import sklearn
from xgboost import XGBClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from optuna.samplers import TPESampler
from optuna.visualization import plot_optimization_history,plot_param_importances

In [17]:

def objective(trial):
    #empty list for training and testing roc_scores
#     print("model number : {}".format(models))
#     models+=1
    test_score = []
    
    #optunas parameters suggestion

    param_grid = {
              'n_estimators': trial.suggest_int('n_estimators', 1,101),
                'max_depth': trial.suggest_int('max_depth',1,6),
              'random_state': 42,
                'criterion':trial.suggest_categorical('criterion', ['gini','entropy'])
                 }
    #cross validation using StratifiedKfold
    
    fold = StratifiedKFold(n_splits=5,shuffle=True,random_state=69)
    i = 0
    for train_id,test_id in fold.split(frature_extractor,train_labels):
        #training and testing data
        print("For Split {}".format(i))
        xtrain ,xtest = frature_extractor[train_id],frature_extractor[test_id]
        ytrain,ytest = train_labels[train_id],train_labels[test_id]
        
        rc = RandomForestClassifier(**param_grid)
        
        rc.fit(xtrain,ytrain)
        
        y_pred = rc.predict(xtrain)
        print("train accuracy:{}".format(accuracy_score(ytrain,y_pred)))
        
        y_pred = rc.predict(xtest)
        print("test accuracy:{}".format(accuracy_score(ytest,y_pred)))
        
        y_pred = rc.predict(frature_extractor_test)
        i+=1
        
        test_score.append(accuracy_score(y_pred,test_labels))
        
        #returning mean score
    return np.mean(test_score)

In [18]:
study = optuna.create_study(direction='maximize',sampler=TPESampler(), study_name='Random_forset_optuna')
study.optimize(objective, n_trials = 100)

[I 2022-12-03 10:35:13,903] A new study created in memory with name: Random_forset_optuna


For Split 0
train accuracy:0.9170506912442397
test accuracy:0.9032258064516129
For Split 1
train accuracy:0.9285714285714286
test accuracy:0.847926267281106
For Split 2
train accuracy:0.9193548387096774
test accuracy:0.8617511520737328
For Split 3
train accuracy:0.9182027649769585
test accuracy:0.8387096774193549
For Split 4


[I 2022-12-03 10:35:16,043] Trial 0 finished with value: 0.8232380952380952 and parameters: {'n_estimators': 75, 'max_depth': 3, 'criterion': 'gini'}. Best is trial 0 with value: 0.8232380952380952.


train accuracy:0.913594470046083
test accuracy:0.8847926267281107
For Split 0
train accuracy:0.9228110599078341
test accuracy:0.9078341013824884
For Split 1
train accuracy:0.9297235023041475
test accuracy:0.8663594470046083
For Split 2
train accuracy:0.9182027649769585
test accuracy:0.8571428571428571
For Split 3
train accuracy:0.9170506912442397
test accuracy:0.8433179723502304
For Split 4


[I 2022-12-03 10:35:18,850] Trial 1 finished with value: 0.8110476190476191 and parameters: {'n_estimators': 85, 'max_depth': 3, 'criterion': 'gini'}. Best is trial 0 with value: 0.8232380952380952.


...

train accuracy:0.956221198156682
test accuracy:0.9170506912442397
For Split 0
train accuracy:0.9735023041474654
test accuracy:0.9078341013824884
For Split 1
train accuracy:0.9746543778801844
test accuracy:0.8755760368663594
For Split 2
train accuracy:0.978110599078341
test accuracy:0.880184331797235
For Split 3
train accuracy:0.9758064516129032
test accuracy:0.847926267281106
For Split 4


[I 2022-12-03 10:40:34,976] Trial 98 finished with value: 0.8579047619047617 and parameters: {'n_estimators': 88, 'max_depth': 5, 'criterion': 'gini'}. Best is trial 64 with value: 0.8651428571428571.


train accuracy:0.9711981566820277
test accuracy:0.9170506912442397
For Split 0
train accuracy:0.9389400921658986
test accuracy:0.9078341013824884
For Split 1
train accuracy:0.9481566820276498
test accuracy:0.8525345622119815
For Split 2


[I 2022-12-03 10:40:35,339] Trial 99 finished with value: 0.7066666666666667 and parameters: {'n_estimators': 7, 'max_depth': 5, 'criterion': 'gini'}. Best is trial 64 with value: 0.8651428571428571.


train accuracy:0.9470046082949308
test accuracy:0.8709677419354839
For Split 3
train accuracy:0.9412442396313364
test accuracy:0.8110599078341014
For Split 4
train accuracy:0.9493087557603687
test accuracy:0.8755760368663594


In [19]:
print("Number of trails : {}".format(len(study.trials)))
print("Best parameters :")
print(study.best_params)
params = study.best_params

Number of trails : 100
Best parameters :
{'n_estimators': 93, 'max_depth': 5, 'criterion': 'gini'}


In [20]:
preds = []
fold = StratifiedKFold(n_splits=5,shuffle=True,random_state=42)
i = 1
for train_id,test_id in fold.split(frature_extractor,train_labels):
        #training and testing data
        print("For Split {}".format(i))
        xtrain ,xtest = frature_extractor[train_id],frature_extractor[test_id]
        ytrain,ytest = train_labels[train_id],train_labels[test_id]
        rc = RandomForestClassifier(**params)
        rc.fit(xtrain,ytrain)
        y_pred = rc.predict(xtrain)
        print("train accuracy:{}".format(accuracy_score(ytrain,y_pred)))
        y_pred = rc.predict(xtest)
        print("test accuracy:{}".format(accuracy_score(ytest,y_pred)))
        y_pred = rc.predict(frature_extractor_test)
        preds.append(y_pred)
        i+=1
preds = np.mean(np.column_stack(preds), axis=1)
new_preds = []
for x in preds:
    if x>=0.:
        new_preds.append(1)
    else:
        new_preds.append(0)
print("Total Test Accuracy:{}".format(accuracy_score(new_preds,test_labels)))

For Split 1
train accuracy:0.9758064516129032
test accuracy:0.8847926267281107
For Split 2
train accuracy:0.9746543778801844
test accuracy:0.9032258064516129
For Split 3
train accuracy:0.9735023041474654
test accuracy:0.8663594470046083
For Split 4
train accuracy:0.978110599078341
test accuracy:0.8755760368663594
For Split 5
train accuracy:0.9758064516129032
test accuracy:0.8986175115207373
Total Test Accuracy:0.44
